In [1]:
import re
import nltk
import emoji
import numpy as np
from nltk.tokenize import word_tokenize
from utils2 import get_dict

In [2]:
# Define the 'tokenize' function that will include the steps previously seen
def tokenize(corpus):
    data = re.sub(r'[,!?;-]+', '.', corpus)
    data = nltk.word_tokenize(data)  # tokenize string to words
    data = [ ch.lower() for ch in data
             if ch.isalpha()
             or ch == '.'
             or emoji.get_emoji_regexp().search(ch)
           ]
    return data

In [4]:
corpus = 'أصدرت دائرة الإسناد الحكومي توجيهات محدّثة بشأن إجراءات التصدي لجائحة كوفيد-19 تنص بعودة جميع الموظفين إلى مقار العمل ابتداءً من يوم الأحد الموافق 5 سبتمبر 2021. وبناءً على ذلك، نلفت إلى ضرورة الالتزام بالنقاط التالية. إننا متشوقون للترحيب بعودتكم جميعاً إلى المكاتب فيما نستأنف العمل بالطاقة الاستيعابية التي عهدناها قبل تفشي الجائحة. وحرصاً على صحتكم وسلامتكم جميعاً باعتبارها في مقدمة أولوياتنا، يطلب من جميع الموظفين والعاملين وعمال عقود الخدمات والتعهيد الالتزام التام بالتوجيهات وإجراءات الصحة والسلامة المتعلقة بكوفيد-19.'

print(f'Corpus:  {corpus}')

# Save tokenized version of corpus into 'words' variable
words = tokenize(corpus)

print(f'Words (tokens):  {words}')

Corpus:  أصدرت دائرة الإسناد الحكومي توجيهات محدّثة بشأن إجراءات التصدي لجائحة كوفيد-19 تنص بعودة جميع الموظفين إلى مقار العمل ابتداءً من يوم الأحد الموافق 5 سبتمبر 2021. وبناءً على ذلك، نلفت إلى ضرورة الالتزام بالنقاط التالية. إننا متشوقون للترحيب بعودتكم جميعاً إلى المكاتب فيما نستأنف العمل بالطاقة الاستيعابية التي عهدناها قبل تفشي الجائحة. وحرصاً على صحتكم وسلامتكم جميعاً باعتبارها في مقدمة أولوياتنا، يطلب من جميع الموظفين والعاملين وعمال عقود الخدمات والتعهيد الالتزام التام بالتوجيهات وإجراءات الصحة والسلامة المتعلقة بكوفيد-19.
Words (tokens):  ['أصدرت', 'دائرة', 'الإسناد', 'الحكومي', 'توجيهات', 'بشأن', 'إجراءات', 'التصدي', 'لجائحة', 'تنص', 'بعودة', 'جميع', 'الموظفين', 'إلى', 'مقار', 'العمل', 'من', 'يوم', 'الأحد', 'الموافق', 'سبتمبر', '.', 'على', 'نلفت', 'إلى', 'ضرورة', 'الالتزام', 'بالنقاط', 'التالية', '.', 'إننا', 'متشوقون', 'للترحيب', 'بعودتكم', 'إلى', 'المكاتب', 'فيما', 'نستأنف', 'العمل', 'بالطاقة', 'الاستيعابية', 'التي', 'عهدناها', 'قبل', 'تفشي', 'الجائحة', '.', 'على', 'صحتكم'

In [5]:
tokenize("أصدرت دائرة الإسناد الحكومي توجيهات محدّثة بشأن إجراءات التصدي لجائحة كوفيد-19 تنص بعودة جميع الموظفين إلى مقار العمل ابتداءً من يوم الأحد الموافق 5 سبتمبر 2021. وبناءً على ذلك، نلفت إلى ضرورة الالتزام بالنقاط التالية.")

['أصدرت',
 'دائرة',
 'الإسناد',
 'الحكومي',
 'توجيهات',
 'بشأن',
 'إجراءات',
 'التصدي',
 'لجائحة',
 'تنص',
 'بعودة',
 'جميع',
 'الموظفين',
 'إلى',
 'مقار',
 'العمل',
 'من',
 'يوم',
 'الأحد',
 'الموافق',
 'سبتمبر',
 '.',
 'على',
 'نلفت',
 'إلى',
 'ضرورة',
 'الالتزام',
 'بالنقاط',
 'التالية',
 '.']

In [7]:
# Define the 'get_windows' function to extract center word and the context words
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

I will use this function to extract context words and center words from a list of tokens. The context and center words will make up the training set that I will use to train the CBOW model.

### Get the Center Word Vector

In [8]:
# Print 'context_words' and 'center_word' for the new corpus with a 'context half-size' of 2
for x, y in get_windows(['أصدرت', 'دائرة', 'الإسناد', 'الحكومي', 'توجيهات', 'بشأن', 'إجراءات', 'التصدي', 'لجائحة', 'تنص', 'بعودة', 'جميع', 'الموظفين', 'إلى', 'مقار', 'العمل', 'من', 'يوم', 'الأحد', 'الموافق', 'سبتمبر', '.', 'على', 'نلفت', 'إلى', 'ضرورة', 'الالتزام', 'بالنقاط', 'التالية', '.'], 2):
    print(f'{x}\t{y}')

['أصدرت', 'دائرة', 'الحكومي', 'توجيهات']	الإسناد
['دائرة', 'الإسناد', 'توجيهات', 'بشأن']	الحكومي
['الإسناد', 'الحكومي', 'بشأن', 'إجراءات']	توجيهات
['الحكومي', 'توجيهات', 'إجراءات', 'التصدي']	بشأن
['توجيهات', 'بشأن', 'التصدي', 'لجائحة']	إجراءات
['بشأن', 'إجراءات', 'لجائحة', 'تنص']	التصدي
['إجراءات', 'التصدي', 'تنص', 'بعودة']	لجائحة
['التصدي', 'لجائحة', 'بعودة', 'جميع']	تنص
['لجائحة', 'تنص', 'جميع', 'الموظفين']	بعودة
['تنص', 'بعودة', 'الموظفين', 'إلى']	جميع
['بعودة', 'جميع', 'إلى', 'مقار']	الموظفين
['جميع', 'الموظفين', 'مقار', 'العمل']	إلى
['الموظفين', 'إلى', 'العمل', 'من']	مقار
['إلى', 'مقار', 'من', 'يوم']	العمل
['مقار', 'العمل', 'يوم', 'الأحد']	من
['العمل', 'من', 'الأحد', 'الموافق']	يوم
['من', 'يوم', 'الموافق', 'سبتمبر']	الأحد
['يوم', 'الأحد', 'سبتمبر', '.']	الموافق
['الأحد', 'الموافق', '.', 'على']	سبتمبر
['الموافق', 'سبتمبر', 'على', 'نلفت']	.
['سبتمبر', '.', 'نلفت', 'إلى']	على
['.', 'على', 'إلى', 'ضرورة']	نلفت
['على', 'نلفت', 'ضرورة', 'الالتزام']	إلى
['نلفت', 'إلى', 'الالتزام', 'بالنق

In [9]:
# Transform words into vectors for the training set
word2Ind, Ind2word = get_dict(words)

In [10]:
word2Ind

{'.': 0,
 'أصدرت': 1,
 'إجراءات': 2,
 'إلى': 3,
 'إننا': 4,
 'الأحد': 5,
 'الإسناد': 6,
 'الاستيعابية': 7,
 'الالتزام': 8,
 'التالية': 9,
 'التام': 10,
 'التصدي': 11,
 'التي': 12,
 'الجائحة': 13,
 'الحكومي': 14,
 'الخدمات': 15,
 'الصحة': 16,
 'العمل': 17,
 'المتعلقة': 18,
 'المكاتب': 19,
 'الموافق': 20,
 'الموظفين': 21,
 'باعتبارها': 22,
 'بالتوجيهات': 23,
 'بالطاقة': 24,
 'بالنقاط': 25,
 'بشأن': 26,
 'بعودة': 27,
 'بعودتكم': 28,
 'تفشي': 29,
 'تنص': 30,
 'توجيهات': 31,
 'جميع': 32,
 'دائرة': 33,
 'سبتمبر': 34,
 'صحتكم': 35,
 'ضرورة': 36,
 'عقود': 37,
 'على': 38,
 'عهدناها': 39,
 'في': 40,
 'فيما': 41,
 'قبل': 42,
 'لجائحة': 43,
 'للترحيب': 44,
 'متشوقون': 45,
 'مقار': 46,
 'مقدمة': 47,
 'من': 48,
 'نستأنف': 49,
 'نلفت': 50,
 'وإجراءات': 51,
 'والتعهيد': 52,
 'والسلامة': 53,
 'والعاملين': 54,
 'وسلامتكم': 55,
 'وعمال': 56,
 'يطلب': 57,
 'يوم': 58}

In [11]:
Ind2word

{0: '.',
 1: 'أصدرت',
 2: 'إجراءات',
 3: 'إلى',
 4: 'إننا',
 5: 'الأحد',
 6: 'الإسناد',
 7: 'الاستيعابية',
 8: 'الالتزام',
 9: 'التالية',
 10: 'التام',
 11: 'التصدي',
 12: 'التي',
 13: 'الجائحة',
 14: 'الحكومي',
 15: 'الخدمات',
 16: 'الصحة',
 17: 'العمل',
 18: 'المتعلقة',
 19: 'المكاتب',
 20: 'الموافق',
 21: 'الموظفين',
 22: 'باعتبارها',
 23: 'بالتوجيهات',
 24: 'بالطاقة',
 25: 'بالنقاط',
 26: 'بشأن',
 27: 'بعودة',
 28: 'بعودتكم',
 29: 'تفشي',
 30: 'تنص',
 31: 'توجيهات',
 32: 'جميع',
 33: 'دائرة',
 34: 'سبتمبر',
 35: 'صحتكم',
 36: 'ضرورة',
 37: 'عقود',
 38: 'على',
 39: 'عهدناها',
 40: 'في',
 41: 'فيما',
 42: 'قبل',
 43: 'لجائحة',
 44: 'للترحيب',
 45: 'متشوقون',
 46: 'مقار',
 47: 'مقدمة',
 48: 'من',
 49: 'نستأنف',
 50: 'نلفت',
 51: 'وإجراءات',
 52: 'والتعهيد',
 53: 'والسلامة',
 54: 'والعاملين',
 55: 'وسلامتكم',
 56: 'وعمال',
 57: 'يطلب',
 58: 'يوم'}

In [16]:
# Verify key '2' within Ind2word dictionary
print("Word which has index 2:  ",Ind2word[2] )

Word which has index 2:   إجراءات


In [17]:
# Save length of word2Ind dictionary into the 'V' variable
V = len(word2Ind)

# Print length of word2Ind dictionary
print("Size of vocabulary: ", V)

Size of vocabulary:  59


In [18]:
# Save index of word 'happy' into the 'n' variable
n = word2Ind['إجراءات']

# Print index of word 'happy'
n

2

In [19]:
# Create One-hot word vectors
# Create vector with the same length as the vocabulary, filled with zeros
center_word_vector = np.zeros(V)

# Print vector
center_word_vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
# Replace element number 'n' with a 1 to have the one-hot word vector.
center_word_vector[n] = 1

In [21]:
# Print vector
center_word_vector

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
# Define the 'word_to_one_hot_vector' function that will include the steps previously seen
def word_to_one_hot_vector(word, word2Ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2Ind[word]] = 1
    return one_hot_vector

In [23]:
# Print output of 'word_to_one_hot_vector' function for word 'happy'
word_to_one_hot_vector('إجراءات', word2Ind, V)

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [24]:
# Print output of 'word_to_one_hot_vector' function for word 'بشأن'
word_to_one_hot_vector('بشأن', word2Ind, V)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

### Get the context word vectors

I will calculate the average of the one-hot vectors representing the individual words for that matter.

In [25]:
# Define list containing context words
context_words = ['أصدرت', 'دائرة', 'الحكومي', 'توجيهات', 'بشأن']

Using Python's list comprehension construct and the `word_to_one_hot_vector` function that I created in the previous section, I can now create a list of one-hot vectors representing each of the context words.

In [26]:
# Create one-hot vectors for each context word using list comprehension
context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]

# Print one-hot vectors for each context word
context_words_vectors

[array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.

In [27]:
# Compute mean of the vectors using numpy
np.mean(context_words_vectors, axis=0)

array([0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0.2, 0. , 0. , 0. , 0. , 0.2, 0. , 0.2, 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. ])

In [28]:
# Define the 'context_words_to_vector' function that will include the steps previously seen
def context_words_to_vector(context_words, word2Ind, V):
    context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
    context_words_vectors = np.mean(context_words_vectors, axis=0)
    return context_words_vectors

In [29]:
# Print output of 'context_words_to_vector' function for context words: 'i', 'am', 'because', 'i'
context_words_to_vector(['أصدرت', 'دائرة', 'الحكومي', 'توجيهات'], word2Ind, V)

array([0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  ,
       0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  ])

**What is the vector representation of the context words "am happy i am"?**

In [30]:
# Print output of 'context_words_to_vector' function for context words: 'دائرة', 'الحكومي', 'توجيهات', 'بشأن'
context_words_to_vector(['دائرة', 'الحكومي', 'توجيهات', 'بشأن'], word2Ind, V)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.25, 0.  ,
       0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  ])


## Build the training set for the CBOW model

In [31]:
# Print corpus
words

['أصدرت',
 'دائرة',
 'الإسناد',
 'الحكومي',
 'توجيهات',
 'بشأن',
 'إجراءات',
 'التصدي',
 'لجائحة',
 'تنص',
 'بعودة',
 'جميع',
 'الموظفين',
 'إلى',
 'مقار',
 'العمل',
 'من',
 'يوم',
 'الأحد',
 'الموافق',
 'سبتمبر',
 '.',
 'على',
 'نلفت',
 'إلى',
 'ضرورة',
 'الالتزام',
 'بالنقاط',
 'التالية',
 '.',
 'إننا',
 'متشوقون',
 'للترحيب',
 'بعودتكم',
 'إلى',
 'المكاتب',
 'فيما',
 'نستأنف',
 'العمل',
 'بالطاقة',
 'الاستيعابية',
 'التي',
 'عهدناها',
 'قبل',
 'تفشي',
 'الجائحة',
 '.',
 'على',
 'صحتكم',
 'وسلامتكم',
 'باعتبارها',
 'في',
 'مقدمة',
 'يطلب',
 'من',
 'جميع',
 'الموظفين',
 'والعاملين',
 'وعمال',
 'عقود',
 'الخدمات',
 'والتعهيد',
 'الالتزام',
 'التام',
 'بالتوجيهات',
 'وإجراءات',
 'الصحة',
 'والسلامة',
 'المتعلقة',
 '.']

I will use the sliding window function (`get_windows`) to extract the context words and center words, and then convert these sets of words into a basic vector representation using `word_to_one_hot_vector` and `context_words_to_vector`.

In [32]:
# Print vectors associated to center and context words for corpus
for context_words, center_word in get_windows(words, 2):  # reminder: 2 is the context half-size
    print(f'Context words:  {context_words} -> {context_words_to_vector(context_words, word2Ind, V)}')
    print(f'Center word:  {center_word} -> {word_to_one_hot_vector(center_word, word2Ind, V)}')
    print()

Context words:  ['أصدرت', 'دائرة', 'الحكومي', 'توجيهات'] -> [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]
Center word:  الإسناد -> [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Context words:  ['دائرة', 'الإسناد', 'توجيهات', 'بشأن'] -> [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.
 0.   0.   0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]
Center word:  الحكومي -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.

Next, I will perform a single iteration of training using a single example to train the CBOW model. 
I will define a Python generator function to make it easier to iterate over a set of examples later on.

In [33]:
# Define the generator function 'get_training_example'
def get_training_example(words, C, word2Ind, V):
    for context_words, center_word in get_windows(words, C):
        yield context_words_to_vector(context_words, word2Ind, V), word_to_one_hot_vector(center_word, word2Ind, V)

The output of this function can be iterated on to get successive context word vectors and center word vectors, as demonstrated in the next cell.

In [34]:
# Print vectors associated to center and context words for corpus using the generator function
for context_words_vector, center_word_vector in get_training_example(words, 2, word2Ind, V):
    print(f'Context words vector:  {context_words_vector}')
    print(f'Center word vector:  {center_word_vector}')
    print()

Context words vector:  [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]
Center word vector:  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Context words vector:  [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.
 0.   0.   0.   0.25 0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]
Center word vector:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

The next phase will be to create the CBOW model